# 0. Settings

In [ ]:
# Change these

locality = "nc-guilford"
verbose = True
clear_checkpoints = False

# 1. Basic setup

In [ ]:
from init_notebooks import setup_environment
setup_environment()

In [ ]:
# import a bunch of stuff

import os
import pandas as pd
from openavmkit.data import (
    load_data,
    enrich_time
)    
from openavmkit.pipeline import (
    NotebookState, 
    set_locality,
    examine_df
)
from openavmkit.utilities.settings import (
    load_settings
)
from openavmkit.cleaning import (
    clean_valid_sales,
    fill_unknown_values
)
from openavmkit.checkpoint import (
    from_checkpoint,
    write_checkpoint,
    delete_checkpoints
)
from openavmkit.time_adjustment import (
    enrich_time_adjustment
)
from openavmkit.horizontal_equity_study import (
    mark_horizontal_equity_clusters_per_model_group
)
from openavmkit.sales_scrutiny_study import (
    run_sales_scrutiny_per_model_group
)

In [ ]:
if 'inited' not in globals():
    nbs: NotebookState = None
    inited = True
nbs = set_locality(nbs, locality)
settings = load_settings()

In [ ]:
if clear_checkpoints:
    delete_checkpoints("1-predict")

## 1.1. Load data

In [ ]:
# load the data
df = load_data(settings)

## 1.2. Fill unknowns in data

In [ ]:
df = fill_unknown_values(df, settings)

In [ ]:
examine_df(df, settings)

# 2. Enrichment

In [ ]:
df = from_checkpoint("2-clean-00-horizontal-equity", mark_horizontal_equity_clusters_per_model_group,
    {
        "df_in": df,
        "settings": settings,
        "verbose": verbose
    }
)

In [ ]:
df = clean_valid_sales(df, settings)

In [ ]:
df = enrich_time(df)

In [ ]:
df = from_checkpoint("2-clean-01-enrich-time-adjustment", enrich_time_adjustment,
    {
        "df_in": df,
        "settings": settings,
        "verbose": verbose
    }
)

In [ ]:
df = from_checkpoint("2-clean-02-sales-scrutiny", run_sales_scrutiny_per_model_group,
    {
        "df_in": df,
        "settings": settings,
        "verbose": verbose
    }
)

In [ ]:
write_checkpoint(df, "2-clean-03-out")